# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [61]:
import pandas as pd
import numpy as np
import sqlite3

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [3]:
go = sqlite3.connect("/Users/pep/Downloads/Great_Outdoors_Data_SQLite/go_sales.sqlite")
g = go.cursor()

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [4]:
product = pd.read_sql_query("SELECT * FROM product", go)
product_type = pd.read_sql_query("SELECT * FROM product_type", go)
product_line = pd.read_sql_query("SELECT * FROM product_line", go)
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go)
sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go)
retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go)
country = pd.read_sql_query("SELECT * FROM country", go)
order_header = pd.read_sql_query("SELECT * FROM order_header", go)
order_details = pd.read_sql_query("SELECT * FROM order_details", go)
target = pd.read_sql_query("SELECT * FROM SALES_TARGETData", go)
returned_item = pd.read_sql_query("SELECT * FROM returned_item", go)
return_reason = pd.read_sql_query("SELECT * FROM return_reason", go)

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [ ]:
product['PRODUCTION_COST'] = pd.to_numeric(product['PRODUCTION_COST'])

prod_cost = (product['PRODUCTION_COST'] > 50) & (product['PRODUCTION_COST'] < 100)

p = product.loc[prod_cost, ['PRODUCT_NAME', 'PRODUCTION_COST']]
p

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [ ]:
product['MARGIN'] = pd.to_numeric(product['MARGIN'])

prod_marge = (product['MARGIN'] > 0.6) | (product['MARGIN'] < 0.2)

product.loc[prod_marge, ['PRODUCT_NAME', 'MARGIN']]


Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [ ]:
betaal_met_francs = country['CURRENCY_NAME'] == 'francs'

country.loc[betaal_met_francs, 'COUNTRY']

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [ ]:
product_marge_filter = pd.to_numeric(product['MARGIN']) > 0.5

product.loc[product_marge_filter, 'INTRODUCTION_DATE'].drop_duplicates()

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 28 rijen)

In [ ]:
adres_filter = (sales_branch['REGION'].notna()) & (sales_branch['ADDRESS2'].notna())

sales_branch.loc[adres_filter, ['ADDRESS1', 'CITY']]

Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [ ]:
dollar_filter = country['CURRENCY_NAME'].str.contains('dollar')

country.loc[dollar_filter, ['COUNTRY']]

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [ ]:
postcode_d_filter = retailer_site['POSTAL_ZONE'].str[0] == 'D'
tweede_adres = retailer_site['ADDRESS2'].notna()

retailer_site.loc[(tweede_adres & postcode_d_filter), ['ADDRESS1', 'ADDRESS2', 'CITY']]

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [ ]:
returned_item['RETURN_CODE'].count()

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [ ]:
sales_branch['REGION'].drop_duplicates().count()

Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [ ]:
product['MARGIN'] = pd.to_numeric(product['MARGIN'])

v1 = np.max(product['MARGIN'])
v2 = np.min(product['MARGIN'])
v3 = np.mean(product['MARGIN'])

v1 # geeft 0.7
v2 # geeft 0.17
v3 # geeft 0.396608695652174

Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [ ]:
sales_branch['ADDRESS2'].notna().count()

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [ ]:
order_details['UNIT_PRICE'] = pd.to_numeric(order_details['UNIT_PRICE'])
order_details['UNIT_SALE_PRICE'] = pd.to_numeric(order_details['UNIT_SALE_PRICE'])
order_details['UNIT_COST'] = pd.to_numeric(order_details['UNIT_COST'])

filter_filter = order_details['UNIT_PRICE'] > order_details['UNIT_SALE_PRICE']

np.mean(order_details['UNIT_PRICE'].where(filter_filter))

Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [ ]:
sales_staff.groupby(['POSITION_EN'])['POSITION_EN'].count()

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [ ]:
result = sales_staff.groupby(['WORK_PHONE'], as_index=False)['WORK_PHONE'].count()

result.loc[result['SALES_STAFF_CODE'] >= 4]

## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [ ]:
nederlandse_klanten = pd.merge(sales_branch, country, left_on='COUNTRY_CODE', how='inner', right_on='COUNTRY_CODE')

land_filter = nederlandse_klanten['COUNTRY'] == 'Netherlands'

nederlandse_klanten.loc[land_filter, ['ADDRESS1', 'CITY']]

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [ ]:
eyewear_merge = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
eyewear_filter = eyewear_merge['PRODUCT_TYPE_EN'] == 'Eyewear'

eyewear_merge.loc[eyewear_filter, ['PRODUCT_NAME']]

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [ ]:
branch_manager_merge = pd.merge(sales_staff, sales_branch, left_on='SALES_BRANCH_CODE', how='outer', right_on='SALES_BRANCH_CODE')
merge_op_een_merge = pd.merge(branch_manager_merge, order_header, left_on='SALES_BRANCH_CODE', how='inner', right_on='SALES_BRANCH_CODE')

branch_manager_filter = (merge_op_een_merge['POSITION_EN'] == 'Branch Manager') & (merge_op_een_merge['SALES_STAFF_CODE_x'] == merge_op_een_merge['SALES_STAFF_CODE_y'])


merge_op_een_merge.loc[branch_manager_filter, ['ADDRESS1', 'FIRST_NAME', 'LAST_NAME']].drop_duplicates()

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

In [ ]:
merge1 = pd.merge(sales_staff, order_header, left_on='SALES_STAFF_CODE', how='inner', right_on='SALES_STAFF_CODE')

merge1_filter = (merge1['POSITION_EN'].str.contains('Manager')) 

merge1.loc[merge1_filter, ['POSITION_EN', 'ORDER_DATE']].drop_duplicates('ORDER_DATE')

##!!!!

Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [5]:
producten_merge = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
producten_filter = pd.merge(producten_merge, order_details, left_on='PRODUCT_NUMBER', how='inner', right_on='PRODUCT_NUMBER')

producten_filter = (producten_filter['PRODUCT_NUMBER'].count() > 750)



Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [ ]:
producten_merge = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
merge2 = pd.merge(producten_merge, order_details, left_on='PRODUCT_NUMBER', how='inner', right_on='PRODUCT_NUMBER')

korting_filter = ((merge2['UNIT_PRICE'].astype(float) - merge2['UNIT_SALE_PRICE'].astype(float)) / merge2['UNIT_PRICE'].astype(float)) > 0.4

merge2.loc[korting_filter, ['PRODUCT_NAME']].drop_duplicates()

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [ ]:
v4 = pd.merge(returned_item, order_details, left_on='ORDER_DETAIL_CODE', how='inner', right_on='ORDER_DETAIL_CODE')
v5 = pd.merge(v4, return_reason, left_on='RETURN_REASON_CODE', how='inner', right_on='RETURN_REASON_CODE')

v5_filter = (v5['RETURN_QUANTITY'].astype(float) / v5['QUANTITY'].astype(float)) > 0.9

v5.loc[v5_filter, ['RETURN_DESCRIPTION_EN']].drop_duplicates()

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [ ]:
v6 = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
df6 = v6.groupby(v6['PRODUCT_TYPE_EN'], as_index=False)['PRODUCT_LINE_CODE'].count()
df6.rename(columns={"PRODUCT_LINE_CODE": "count"})

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [ ]:
v7 = pd.merge(retailer_site, country, left_on="COUNTRY_CODE", how="inner", right_on="COUNTRY_CODE")
df7 = v7.groupby(v7['COUNTRY'], as_index=False)['COUNTRY_CODE'].count()
df7.rename(columns={"COUNTRY_CODE": "count"})

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [ ]:
v8 = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', right_on='PRODUCT_TYPE_CODE', how='inner')
v9 = pd.merge(v8, order_details, left_on="PRODUCT_NUMBER", how="inner", right_on="PRODUCT_NUMBER")

v9.groupby(['PRODUCT_NAME','QUANTITY'], as_index=False).agg(
    avg_price=('UNIT_COST', 'mean'),

)

Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

In [ ]:
df1 = pd.merge(sales_staff, sales_branch, left_on='SALES_BRANCH_CODE', how='inner', right_on='SALES_BRANCH_CODE')
df2 = pd.merge(df1, order_header, right_on='SALES_STAFF_CODE', left_on='SALES_STAFF_CODE', how='inner')
filter_ams = (df2['CITY'] == 'Amsterdam')# & (df2['ORDER_DATE'].str.contains('2016'))
df2.loc[filter_ams, ['FIRST_NAME', 'LAST_NAME']].drop_duplicates()

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [6]:
from datetime import date
date.today().year

2024

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [7]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 